# Processing Pipeline Visualizations

In [ ]:
import json
from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp

import pingouin as pg

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticks
import seaborn as sns

from fau_colors import cmaps
from fau_colors import colors

from biopsykit.signals.imu.wear_detection import WearDetection
from biopsykit.sleep.sleep_processing_pipeline import predict_pipeline_acceleration
from biopsykit.sleep.plotting import sleep_imu_plot


from carwatch_analysis.datasets import CarWatchDatasetRaw
from carwatch_analysis.plotting import static_moment_plot

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (14, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

pg.options["round"] = 4

tight_kwargs = {"rect": (0, 0, 0.825, 1.0), "pad": 0.2}
legend_kwargs = {"loc": "upper right", "bbox_to_anchor": (1.0, 1.0)}

save_fig = True

palette

## Setup Paths

In [ ]:
deploy_type = "local"

In [ ]:
# build path to data folder
config_dict = json.load(Path("../../../config.json").open(encoding="utf-8"))
base_path = Path("..").joinpath(config_dict[deploy_type]["base_path"])
base_path

In [ ]:
export_path = Path("../../exports")
result_path = Path("../../results")

# plot_path_car = result_path.joinpath("plots/presentation/example_car")
plot_path_sleep = result_path.joinpath("plots/presentation/sleep_processing_pipeline")
plot_path_imu = result_path.joinpath("plots/presentation/imu_feature_extraction_pipeline")

for path in [plot_path_sleep, plot_path_imu]:
    path.mkdir(exist_ok=True, parents=True)

In [ ]:
dataset = CarWatchDatasetRaw(base_path)
dataset

## Sleep/Wake Detection

In [ ]:
data = dataset.get_subset(subject="AK12I", night=1).imu
fs = dataset.sampling_rate
data = data.between_time("23:30", "07:00")

In [ ]:
sleep_results = predict_pipeline_acceleration(data, dataset.sampling_rate)
sleep_endpoints = sleep_results["sleep_endpoints"]

In [ ]:
major_wear_block = WearDetection.get_major_wear_block(sleep_results["wear_detection"])

In [ ]:
wear_start = major_wear_block[0]
wear_stop = major_wear_block[1]

### Raw ACC

In [ ]:
fig, ax = plt.subplots()
sleep_imu_plot(data, datastreams=["acc"], downsample_factor=10 * fs, ax=ax)

ax.legend().remove()
fig.legend(**legend_kwargs)
fig.tight_layout(**tight_kwargs)

if save_fig:
    fig.savefig(plot_path_sleep.joinpath("img_00_sleep_raw.pdf"), transparent=True)

### Wear Detection

In [ ]:
fig, ax = plt.subplots()
sleep_imu_plot(data, datastreams=["acc"], downsample_factor=10 * fs, ax=ax)

ax.axvspan(wear_start, wear_stop, color=colors.med, alpha=0.1, label="Wear Period")

ax.legend().remove()
fig.legend(**legend_kwargs)
fig.tight_layout(**tight_kwargs)

if save_fig:
    fig.savefig(plot_path_sleep.joinpath("img_01_sleep_wear_detection.pdf"), transparent=True)

### Bed Interval


In [ ]:
data_cut = data.loc[wear_start : (wear_stop - pd.Timedelta("1:30:00"))]

In [ ]:
sleep_results = predict_pipeline_acceleration(data_cut, fs, convert_to_g=True)
sleep_endpoints = sleep_results["sleep_endpoints"]

In [ ]:
fig, ax = plt.subplots()
sleep_imu_plot(
    data,
    datastreams=["acc"],
    sleep_endpoints=sleep_endpoints,
    downsample_factor=10 * fs,
    ax=ax,
    plot_sleep_onset=False,
    plot_wake_onset=False,
    plot_sleep_wake=False,
)

ax.axvspan(wear_start, wear_stop, color=colors.med, alpha=0.1, label="Wear Period", zorder=0)

ax.set_title(None)
ax.legend().remove()
fig.legend(**legend_kwargs)
fig.tight_layout(**tight_kwargs)

if save_fig:
    fig.savefig(plot_path_sleep.joinpath("img_02_sleep_mrp.pdf"), transparent=True)

### Sleep/Wake Detection

In [ ]:
fig, ax = plt.subplots()
sleep_imu_plot(data, datastreams=["acc"], sleep_endpoints=sleep_endpoints, downsample_factor=10 * fs, ax=ax)

ax.axvspan(
    major_wear_block[0],
    pd.Timestamp(sleep_endpoints["sleep_onset"]),
    color=colors.med,
    alpha=0.1,
    label="Wear Period",
    zorder=0,
)
ax.axvspan(pd.Timestamp(sleep_endpoints["wake_onset"]), major_wear_block[1], color=colors.med, alpha=0.1, zorder=0)

ax.set_title(None)
ax.legend().remove()
fig.legend(**legend_kwargs)
fig.tight_layout(**tight_kwargs)

if save_fig:
    fig.savefig(plot_path_sleep.joinpath("img_03_sleep_total.pdf"), transparent=True)

## Sleep/Wake Detection – Subplots

In [ ]:
data = dataset.get_subset(subject="AK12I", night=1).imu
fs = dataset.sampling_rate
data = data.between_time("23:30", "06:00")

In [ ]:
sleep_results = predict_pipeline_acceleration(data, fs, convert_to_g=True)
sleep_endpoints = sleep_results["sleep_endpoints"]

In [ ]:
major_wear_block = WearDetection.get_major_wear_block(sleep_results["wear_detection"])
wear_start = major_wear_block[0]
wear_stop = major_wear_block[1]

In [ ]:
from matplotlib.dates import DateFormatter

fig, axs = plt.subplots(figsize=(22, 5), ncols=4, sharey=True)

sleep_imu_plot(data, datastreams=["acc"], downsample_factor=10 * fs, ax=axs[0])

sleep_imu_plot(data, datastreams=["acc"], downsample_factor=10 * fs, ax=axs[1])
axs[1].axvspan(wear_start, wear_stop, color=colors.med, alpha=0.1, label="Wear Period")

sleep_imu_plot(
    data,
    datastreams=["acc"],
    sleep_endpoints=sleep_endpoints,
    downsample_factor=10 * fs,
    ax=axs[2],
    plot_sleep_onset=False,
    plot_wake_onset=False,
    plot_sleep_wake=False,
)
axs[2].axvspan(wear_start, wear_stop, color=colors.med, alpha=0.1, label="Wear Period", zorder=0)

sleep_imu_plot(data, datastreams=["acc"], sleep_endpoints=sleep_endpoints, downsample_factor=10 * fs, ax=axs[3])
axs[3].axvspan(
    major_wear_block[0],
    pd.Timestamp(sleep_endpoints["sleep_onset"]),
    color=colors.med,
    alpha=0.1,
    label="Wear Period",
    zorder=0,
)
axs[3].axvspan(pd.Timestamp(sleep_endpoints["wake_onset"]), major_wear_block[1], color=colors.med, alpha=0.1, zorder=0)

for ax in axs:
    ax.xaxis.set_major_formatter(DateFormatter("%H:%M"))
    h, l = ax.get_legend_handles_labels()
    ax.legend().remove()
    ax.set_title(None)

fig.legend(h, l, **legend_kwargs)
fig.tight_layout(pad=0.2, rect=(0, 0, 0.89, 1.0))
if save_fig:
    fig.savefig(plot_path_sleep.joinpath("img_sleep_pipeline_total.pdf"), transparent=True)

## Static Periods Plot

### Whole Night

In [ ]:
data = dataset.get_subset(subject="AK12I", night=1).imu
fs = dataset.sampling_rate
data = data.between_time("23:30", "07:00")

In [ ]:
sleep_results = predict_pipeline_acceleration(data, fs, convert_to_g=True)
sleep_endpoints = sleep_results["sleep_endpoints"]

In [ ]:
major_wear_block = WearDetection.get_major_wear_block(sleep_results["wear_detection"])

In [ ]:
wear_start = major_wear_block[0]
wear_stop = major_wear_block[1]

In [ ]:
fig, ax = plt.subplots()
sleep_imu_plot(data, datastreams=["acc"], sleep_endpoints=sleep_endpoints, downsample_factor=10 * fs, ax=ax)
ax.set_title(None)
fig.tight_layout()

### Raw

In [ ]:
data_cut = data.loc[wear_start : (wear_stop - pd.Timedelta("1:30:00"))]

sleep_results = predict_pipeline_acceleration(data_cut, fs, convert_to_g=True)
sleep_endpoints = sleep_results["sleep_endpoints"]

In [ ]:
wake_onset = pd.to_datetime(sleep_endpoints["wake_onset"])

In [ ]:
data_30 = data.loc[(wake_onset - pd.Timedelta("0:30:00")) : wake_onset]

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
fig, ax = static_moment_plot(data_30, wake_onset, ax=ax)
fig.savefig(plot_path_imu.joinpath("img_00_imu_feature_extraction_raw.pdf"), transparent=True)

### Static Moments

In [ ]:
static_moments = bp.signals.imu.static_moment_detection.find_static_moments(
    data_30.filter(like="gyr"), threshold=100, window_sec=5, overlap_percent=0.9, sampling_rate=fs
)

# filter static moments longer than 60s
static_moments_mask = static_moments.diff(axis=1)["end"] > 60 * fs
static_moments = static_moments.loc[static_moments_mask]

# convert indices to timestamps
static_moments = static_moments.applymap(lambda val: data_30.index[val])
static_moments

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
fig, ax = static_moment_plot(data_30, wake_onset, static_moments=static_moments, ax=ax)

fig.savefig(plot_path_imu.joinpath("img_01_imu_feature_extraction_static_moments.pdf"), transparent=True)

### Static Moment Features

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
fig, ax = static_moment_plot(
    data_30, wake_onset, static_moments=static_moments, plot_static_moment_features=True, ax=ax
)

fig.savefig(plot_path_imu.joinpath("img_02_imu_feature_extraction_features.pdf"), transparent=True)